### AB Model from spec

Package installation

In [ ]:
# !pip3 install seaborn
# !pip3 install python-statemachine
# !pip3 install mesa
# !pip3 install transitions
# !pip3 install scipy
# !pip3 install cufflinks
# !pip3 install graphviz pygraphviz
# !pip3 install graphviz
# !pip3 install transitions[diagrams]
# !pip3 uninstall mesa

Package importation

In [1]:
# imports
import os
import seaborn as sns
from random import choice
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import mesa
from mesa import Agent, Model
from mesa.time import RandomActivation, RandomActivationByType, SimultaneousActivation
from mesa.datacollection import DataCollector
from matplotlib import pyplot as plt, patches
import scipy.stats as ss
import cufflinks as cf
cf.go_offline()
from plotly.offline import iplot
from transitions import Machine
import random
from transitions.extensions import GraphMachine
import graphviz
import timeit
import logging
# os.environ["PATH"] += os.pathsep + '/Users/ia329/homebrew/bin' # for graphviz

Model component importation

In [15]:
import model_config as cfg
import analysis.helper as helper
from EV.agent import EV, ChargeStation
import EV.model as model
from EV.statemachine import EVSM, LSM
from EV.model import get_evs_charge, get_evs_charge_level, get_evs_active, get_evs_queue, get_evs_travel, get_evs_not_idle, get_active_chargestations, get_eod_evs_socs, get_evs_destinations, get_ev_distance_covered
# old:  get_evs_charging,

State diagram

### Model

Model parameters

In [3]:
ticks =  48 # 3600 ticks = 3600 seconds = 1 hour
# One month = 30 days = 720 hours
# One year = 365 days = 8760 hours
no_evs = 2
# no of css must be fixed
no_css = 5

Run Model

In [11]:
model_run = model.EVModel(ticks=ticks, no_evs=no_evs, no_css=no_css)
for i in range(ticks):
    model_run.step()


EV info: ID: 0, destination name: friend_2, journey type: Urban, max_battery: 63, energy consumption rate: 0.2, speed: 10, State: Idle.
EV info (Cont'd): Start time: 8, distance goal: 90, soc usage threshold: 31.5, charge prop 0.5, location: City_D.

EV info: ID: 1, destination name: market, journey type: Urban, max_battery: 48, energy consumption rate: 0.2, speed: 10, State: Idle.
EV info (Cont'd): Start time: 11, distance goal: 40, soc usage threshold: 24.0, charge prop 0.5, location: City_D.

CP info: ID: 2, initialized. Charge rate: 7.5 kW.

CP info: ID: 3, initialized. Charge rate: 7.5 kW.

CP info: ID: 4, initialized. Charge rate: 7.5 kW.

CP info: ID: 5, initialized. Charge rate: 7.5 kW.

CP info: ID: 6, initialized. Charge rate: 7.5 kW.
Charging Station: 2 is at checkpoint: 40 miles.
Charging Station: 3 is at checkpoint: 80 miles.
Charging Station: 4 is at checkpoint: 120 miles.
Charging Station: 5 is at checkpoint: 160 miles.
Charging Station: 6 is at checkpoint: 200 miles.



In [12]:
run_stats = model_run.datacollector.get_model_vars_dataframe()
print(run_stats)

    EVs Charging  EVs Activated  EVs Travelling  EVs Queued  EVs Dead  \
0              0              2               0           0         0   
1              0              2               0           0         0   
2              0              2               0           0         0   
3              0              2               0           0         0   
4              0              2               0           0         0   
5              0              2               0           0         0   
6              0              2               0           0         0   
7              0              2               0           0         0   
8              0              2               1           0         0   
9              0              2               1           0         0   
10             0              2               1           0         0   
11             0              2               2           0         0   
12             0              2               2    

In [ ]:
# a = 26 % 24
# print(a) 

Export results to CSV file

In [13]:

model_run.datacollector.get_model_vars_dataframe().to_csv(cfg.DATA_PATH/)
model_run.datacollector.get_model_vars_dataframe().to_csv('/Users/ia329/Projects/ec4d/analysis/06_03_5EV_agent_model_output.csv')

In [ ]:
# def unpack_datacollector(dc):
#     """
#     Unpacks the data from a DataCollector instance in Mesa.
    
#     Args:
#         dc (DataCollector): A DataCollector instance in Mesa.
    
#     Returns:
#         A dictionary with the keys as the variable names and the values as the lists of data for each variable.
#     """
#     data = {}
#     for varname, var in dc.model_vars.items():
#         data[varname] = var.get_data()
#     return data

### Data import and preprocessing

Helper functions. May eventually move to external module

In [14]:
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd

Data import

In [ ]:
data = pd.read_csv('06_03_5EV_agent_model_output.csv')

In [ ]:
data.info()
# data.head()

In [ ]:
a = helper.unpack_and_join(data, 'EVs Charge Level')
# print(a)

In [ ]:
# # EV charge level per EV per timestep - 20 EVs unpacked
# newdf = a[['EVs Charge Level', 'EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4', 'EVs Charge Level_unpacked_5', 'EVs Charge Level_unpacked_6', 'EVs Charge Level_unpacked_7', 'EVs Charge Level_unpacked_8', 'EVs Charge Level_unpacked_9', 'EVs Charge Level_unpacked_10', 'EVs Charge Level_unpacked_11', 'EVs Charge Level_unpacked_12', 'EVs Charge Level_unpacked_13', 'EVs Charge Level_unpacked_14', 'EVs Charge Level_unpacked_15', 'EVs Charge Level_unpacked_16', 'EVs Charge Level_unpacked_17', 'EVs Charge Level_unpacked_18', 'EVs Charge Level_unpacked_19']]
# newdf.head()

# EV charge level per EV per timestep - 20 EVs unpacked
newdf = a[['EVs Charge Level', 'EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4']]
newdf.head()

In [ ]:
add_time_col(newdf)
newdf.head()

### Batching

In [ ]:
from mesa.batchrunner import BatchRunner

EVcounts = (100,500,1000)
cpcounts = (1,2)
tickcounts = (24,48)
model_reporters={'EVs Charged': get_evs_charged,
                'EVs Activated': get_evs_active,
                'EVs Travelling': get_evs_travel,
                'EVs Charge Level': get_evs_charge_level,
                'EVs Currently charging': get_evs_charging,
                'EVs Not Idle': get_evs_not_idle,
                'EOD Battery SOC': get_eod_evs_socs,
                'EVs Destinations': get_evs_destinations,
                }
# parameters = {"no_evs": range(1000,20000,3000), "no_cps": 1}
parameters = {"no_evs": EVcounts, "no_cps": cpcounts, "ticks": tickcounts}
batch_run = BatchRunner(model.EVModel, parameters, max_steps=24, iterations=1, model_reporters= model_reporters) #iterations=1
batch_run.run_all()

In [ ]:
# batch_df = batch_run.get_model_vars_dataframe()

In [ ]:
# print(batch_df)

### Visualisations

Scatter plot

In [ ]:
# plot_data_lines(newdf, 'Timestep', ['EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4'])

### Scrapbook